<a href="https://colab.research.google.com/github/tahoorak-s/IntegriScan/blob/main/IntegriScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**IntegriScan**: File Integrity Checking By Comparing and Calculating Hash Values

###Overview:
IntegriScan is a Python-based cybersecurity tool that ensures the integrity of files in a directory. It works by calculating cryptographic <B>SHA-256</B> hash values for each file and comparing them with previously stored values to detect modifications, deletions, or unauthorized additions.

###Tools Used:
- Language: Python
- Libraries: hashlib, os, json, zipfile
- Environment: Google Colab

###How It Works:
1. A folder (uploaded as a ZIP) is scanned.
2. Each file’s hash is calculated and stored in `hashes.json`.
3. On future runs, the script compares the current file states with the previous hashes.
4. It reports any changes found: modified, deleted, or added files.


---





###STEP 1: Upload a ZIP file containing the files you want to check

In [21]:
from google.colab import files

uploaded = files.upload()


Saving Downloads.zip to Downloads.zip


###STEP 2: Unzip the Folder

In [17]:
import zipfile
import os

zip_file_name = "Downloads.zip"
output_dir = "/content/scan_folder"

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"Files extracted to: {output_dir}")


Files extracted to: /content/scan_folder


### STEP 3: Generate Hashes for All Files

In [22]:
import hashlib
import json

def calculate_hash(file_path):
    sha256 = hashlib.sha256()
    try:
        with open(file_path, 'rb') as f:
            while chunk := f.read(4096):
                sha256.update(chunk)
        return sha256.hexdigest()
    except Exception as e:
        return f"ERROR: {e}"

def generate_hashes(directory):
    file_hashes = {}
    for root, _, files in os.walk(directory):
        for name in files:
            full_path = os.path.join(root, name)
            file_hashes[full_path] = calculate_hash(full_path)
    return file_hashes

def save_hashes(hashes, filename="hashes.json"):
    with open(filename, 'w') as f:
        json.dump(hashes, f, indent=4)

# Run the hash generation
hashes = generate_hashes(output_dir)
save_hashes(hashes)
print("✅ Hashes saved to hashes.json")


✅ Hashes saved to hashes.json


###STEP 4: Compare with Previous Hashes (to detect changes)


In [23]:
def load_hashes(filename="hashes.json"):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print("⚠️ No previous hash file found.")
        return {}

def compare_hashes(old_hashes, new_hashes):
    modified = []
    deleted = []
    added = []

    old_files = set(old_hashes.keys())
    new_files = set(new_hashes.keys())

    for file in old_files:
        if file not in new_files:
            deleted.append(file)
        elif old_hashes[file] != new_hashes[file]:
            modified.append(file)

    for file in new_files - old_files:
        added.append(file)

    return modified, deleted, added



In [25]:
def exec_result():
  # Simulate a second scan (you can rerun this after modifying files and re-uploading)
  new_hashes = generate_hashes(output_dir)
  old_hashes = load_hashes()

  modified, deleted, added = compare_hashes(old_hashes, new_hashes)

  # Print the results
  print("\n🔍 File Change Report:")
  print("📝 Modified Files:")
  print("\n".join(modified) if modified else " - None")
  print("❌ Deleted Files:")
  print("\n".join(deleted) if deleted else " - None")
  print("➕ New Files:")
  print("\n".join(added) if added else " - None")

####CASE I: NO CHANGES TO THE FILES

In [26]:
exec_result()


🔍 File Change Report:
📝 Modified Files:
 - None
❌ Deleted Files:
 - None
➕ New Files:
 - None


####CASE II: A FILE IS DELETED

In [27]:
exec_result()


🔍 File Change Report:
📝 Modified Files:
 - None
❌ Deleted Files:
/content/scan_folder/business presentation.docx
➕ New Files:
 - None


####CASE III: NEW FILE IS ADDED TO THE FOLDER

In [28]:
exec_result()


🔍 File Change Report:
📝 Modified Files:
 - None
❌ Deleted Files:
/content/scan_folder/business presentation.docx
➕ New Files:
/content/scan_folder/untitled


####CASE IV: TO DETECT A MODIFIED FILE IN THE FOLDER


<U>Step 1: Create a Test Folder with Files</U>

In [29]:
import os

test_dir = "/content/test_folder"
os.makedirs(test_dir, exist_ok=True)

# Create sample files
with open(f"{test_dir}/file1.txt", "w") as f:
    f.write("Original content.")

with open(f"{test_dir}/file2.txt", "w") as f:
    f.write("This file will be changed.")


<U>Step 2: Generate Baseline Hashes

In [14]:
hashes = generate_hashes(test_dir)
save_hashes(hashes, "hash.json")
print("✅ Baseline hashes generated.")


✅ Baseline hashes generated.


<U>Step 3: Modify One File (simulate tampering)

In [15]:
# Modify file2.txt
with open(f"{test_dir}/file2.txt", "a") as f:
    f.write("\nThis is malicious code!")

print("⚠️ file2.txt has been modified.")


⚠️ file2.txt has been modified.


<U>Step 4: Compare with Baseline

In [16]:
new_hashes = generate_hashes(test_dir)
old_hashes = load_hashes("hash.json")

modified, deleted, added = compare_hashes(old_hashes, new_hashes)

print("\n🔍 File Change Report:")
print("📝 Modified Files:")
print("\n".join(modified) if modified else " - None")
print("❌ Deleted Files:")
print("\n".join(deleted) if deleted else " - None")
print("➕ New Files:")
print("\n".join(added) if added else " - None")



🔍 File Change Report:
📝 Modified Files:
/content/test_folder/file2.txt
❌ Deleted Files:
 - None
➕ New Files:
 - None


---

###Outcome:
The script effectively monitors file integrity and provides an essential baseline for intrusion detection or digital forensics.